# [Quantum Phase Estimation Code used from IBM's Qiskit Tutorials](https://qiskit.org/textbook/ch-algorithms/quantum-phase-estimation.html)
## Code adapted for Cirq

In [1]:
import cirq
import random
import matplotlib.pyplot as plt
import numpy as np
import math

In [2]:
def qft_dagger_cirq(qc, qubits, n):
    for qubit in range(n//2):
        qc.append(cirq.SWAP(qubits[qubit], qubits[n-qubit-1]))
    for j in range(n):
        for m in range(j):
            qc.append((cirq.CZ**(-1/2**(j-m)))(qubits[m],qubits[j]))
        qc.append(cirq.H(qubits[j]))

In [16]:
def generalised_qpe_cirq(amt_estimation_qubits, angle, shots = 10000):
    # Create and set up circuit
    qubits = cirq.LineQubit.range(amt_estimation_qubits+1)
    circuit = cirq.Circuit()
    
    # Apply H-Gates to counting qubits:
    for qubit in range(amt_estimation_qubits):
        circuit.append(cirq.H(qubits[qubit]))

    #print(circuit)    
        
    # Prepare our eigenstate |psi>:
    circuit.append(cirq.X(qubits[amt_estimation_qubits]))

    #print(circuit) 
    
    repetitions = 1
    for counting_qubit in range(amt_estimation_qubits):
        for i in range(repetitions):
            circuit.append((cirq.CZ**((angle/math.pi)))(qubits[counting_qubit],qubits[amt_estimation_qubits]));
        repetitions *= 2
        
    #print(circuit) 

    # Do the inverse QFT:
    qft_dagger_cirq(circuit, qubits, amt_estimation_qubits)
    
    # Measure of course!
    circuit.append(cirq.measure(*qubits[:-1], key='m'))
        
    simulator = cirq.Simulator()
    results = simulator.run(circuit , repetitions = shots)
    theta_estimates = np.sum(2 ** np.arange(amt_estimation_qubits) * results.measurements['m'], axis=1) / 2**amt_estimation_qubits
    
    print(theta_estimates)
    counts = dict()
    for i in theta_estimates:
        counts[i] = counts.get(i, 0) + 1
    print(counts)
    
    unique,pos = np.unique(theta_estimates,return_inverse=True)
    counts = np.bincount(pos)
    maxpos = counts.argmax()
    print(unique[maxpos],counts[maxpos])
    
#generalised_qpe_cirq(7,(13*math.pi/9))
generalised_qpe_cirq(7,(13*math.pi/9))

[0.71875   0.7265625 0.734375  ... 0.71875   0.71875   0.71875  ]
{0.71875: 5010, 0.7265625: 3207, 0.734375: 372, 0.765625: 32, 0.7421875: 153, 0.703125: 164, 0.7734375: 28, 0.671875: 19, 0.7109375: 454, 0.8828125: 5, 0.6640625: 19, 0.6953125: 78, 0.7890625: 11, 0.3515625: 1, 0.640625: 8, 0.6875: 38, 0.75: 88, 0.65625: 10, 0.59375: 3, 0.7578125: 45, 0.328125: 3, 0.5234375: 3, 0.6171875: 3, 0.6796875: 40, 0.84375: 6, 0.6328125: 9, 0.359375: 1, 0.4296875: 2, 0.515625: 2, 0.5546875: 6, 0.796875: 13, 0.8671875: 4, 0.6015625: 5, 0.09375: 2, 0.8046875: 10, 0.1796875: 2, 0.5625: 4, 0.78125: 12, 0.828125: 7, 0.8203125: 12, 0.859375: 3, 0.875: 3, 0.4609375: 1, 0.578125: 4, 0.984375: 2, 0.96875: 3, 0.3359375: 1, 0.890625: 5, 0.625: 8, 0.8359375: 5, 0.171875: 3, 0.8515625: 3, 0.6484375: 12, 0.453125: 2, 0.90625: 1, 0.3828125: 2, 0.53125: 5, 0.1953125: 1, 0.03125: 1, 0.953125: 2, 0.34375: 1, 0.8125: 3, 0.0078125: 1, 0.9609375: 1, 0.0234375: 1, 0.609375: 2, 0.1015625: 1, 0.921875: 1, 0.9140625: 3, 